In [ ]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#Comment this lines if you have this stuff already installed
#!(yes |pip install geopandas)
#!(yes |pip install descartes)
#!(yes |conda install -c conda-forge geoplot)
#!(yes | pip install plotly)
import plotly.express as px

%config IPCompleter.greedy=True
%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.layers import Dropout

In [ ]:
trainCSV = pd.read_csv('./forHiper')

In [ ]:
x = trainCSV.drop(['id', 'text', 'keyword', 'location','target'], axis=1)
y = trainCSV.pop('target')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.01, random_state=42)

In [ ]:
#Get best optimizer
def build_model(optimizer):
  model = Sequential()
  model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['acc'])
  return model

parameters = {'batch_size': [8,16,32],
             'epochs':[100,200,500],
             'optimizer': ['adadelta', 'rmsprop', 'adam']}

estimator = KerasClassifier(build_fn=build_model, verbose=0)
grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, scoring='accuracy', cv=10)
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
# Get best layer config
def build_model(l1, l2):
  model = Sequential()
  model.add(Dense(l1, input_shape=(X_train.shape[1],), activation='relu'))
  # model.add(Dropout(0.1))
  model.add(Dense(l2, activation='relu'))
  # model.add(Dropout(0.1))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])
  return model

parameters = {'l1':[16,32,64,128,256],
               'l2':[16,32,64,128]}

estimator = KerasClassifier(build_fn=build_model, verbose=0, batch_size=16, epochs=100)
grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, scoring='accuracy', cv=10)
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
#Get best dropout config
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import GridSearchCV
def build_model(d1, d2):
    model = Sequential()
    model.add(Dense(256, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(d1))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(d2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['acc'])
    return model

parameters = {'d1':[0.01,0.1,0.2,0.25],
              'd2':[0.01,0.1,0.2,0.25]}

estimator = KerasClassifier(build_fn=build_model, verbose=0, batch_size=16, epochs=200)
grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, scoring='accuracy', cv=10)
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
# (64, 16), (64, 32), (128, 16) va bien
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu')) # 256
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu')) # 128
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # adadelta

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=10)

In [ ]:
results = model.evaluate(X_test, y_test, batch_size=10)
print("test loss, test acc:", results)

In [ ]:
testCSV = pd.read_csv('./testWithFeatures.csv')
x_predict = testCSV.drop(['id', 'text', 'keyword', 'location'], axis=1)
x_predict.head()

In [ ]:
# (0.2, 0.01) -> 0.8293963074684143
predictions = model.predict_classes(x_predict)
print("predictions shape:", predictions.shape)

In [ ]:
b = predictions.ravel()
pd.Series(b).value_counts()

In [ ]:
submissionDf = pd.read_csv('./sample_submission.csv')
submissionDf['target'] = b
submissionDf['target'].value_counts()

In [ ]:
submissionDf.head(10)

In [ ]:
submissionDf.to_csv('submission.csv', index = False)